In [11]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import ot
import sys
import scanpy as sc
import GEOparse
import anndata as ad
import os
import tarfile
import scipy.io
from scipy import io
from scipy.sparse import issparse
from scipy.sparse import csr_matrix

In [31]:
adata = ad.read_h5ad('/workspace/ImputationOT/data/PBMC.merged.h5ad')
print(adata)

/opt/conda/lib/python3.10/site-packages/anndata/compat/__init__.py:311: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/opt/conda/lib/python3.10/site-packages/anndata/compat/__init__.py:311: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


AnnData object with n_obs × n_vars = 15476 × 1199
    obs: 'Sample', 'n_counts', 'n_genes', 'batch', 'louvain', 'anno', 'Method', 'Cell type'
    var: 'GeneName-0', 'GeneID-0', 'GeneName-1', 'GeneID-1', 'means', 'dispersions', 'dispersions_norm'
    uns: 'Cell type_colors', 'Method_colors', 'Sample_colors', 'anno_colors', 'louvain', 'louvain_colors', 'neighbors', 'rank_genes_groups'
    obsm: 'X_pca', 'X_umap', 'X_tsne'
    obsp: 'distances', 'connectivities'


In [32]:
print(len(adata.obs['Cell type'].unique()))
print(adata.obs['Cell type'])

9
index
data_3p-AAACCTGAGCATCATC-0                         B cell
data_3p-AAACCTGAGCTAGTGG-0                     CD4 T cell
data_3p-AAACCTGCACATTAGC-0                     CD4 T cell
data_3p-AAACCTGCACTGTTAG-0                  Monocyte_CD14
data_3p-AAACCTGCATAGTAAG-0                  Monocyte_CD14
                                         ...             
data_5p-TTTGTCATCCACGTTC-1                  Monocyte_CD14
data_5p-TTTGTCATCGCGTAGC-1                         B cell
data_5p-TTTGTCATCTTAACCT-1                     CD8 T cell
data_5p-TTTGTCATCTTACCGC-1    Plasmacytoid dendritic cell
data_5p-TTTGTCATCTTGTTTG-1                     CD8 T cell
Name: Cell type, Length: 15476, dtype: category
Categories (9, object): ['B cell', 'CD4 T cell', 'CD8 T cell', 'Hematopoietic stem cell', ..., 'Monocyte_CD14', 'Monocyte_FCGR3A', 'NK cell', 'Plasmacytoid dendritic cell']


In [34]:
print(len(adata.obs['batch'] == 1))
print(adata.obs['batch'])

15476
index
data_3p-AAACCTGAGCATCATC-0    0
data_3p-AAACCTGAGCTAGTGG-0    0
data_3p-AAACCTGCACATTAGC-0    0
data_3p-AAACCTGCACTGTTAG-0    0
data_3p-AAACCTGCATAGTAAG-0    0
                             ..
data_5p-TTTGTCATCCACGTTC-1    1
data_5p-TTTGTCATCGCGTAGC-1    1
data_5p-TTTGTCATCTTAACCT-1    1
data_5p-TTTGTCATCTTACCGC-1    1
data_5p-TTTGTCATCTTGTTTG-1    1
Name: batch, Length: 15476, dtype: category
Categories (2, object): ['0', '1']


In [35]:
print(len(adata.obs['louvain'].unique()))
print(adata.obs['louvain'])

33
index
data_3p-AAACCTGAGCATCATC-0     9
data_3p-AAACCTGAGCTAGTGG-0     5
data_3p-AAACCTGCACATTAGC-0     1
data_3p-AAACCTGCACTGTTAG-0     0
data_3p-AAACCTGCATAGTAAG-0     0
                              ..
data_5p-TTTGTCATCCACGTTC-1     3
data_5p-TTTGTCATCGCGTAGC-1    10
data_5p-TTTGTCATCTTAACCT-1    16
data_5p-TTTGTCATCTTACCGC-1    28
data_5p-TTTGTCATCTTGTTTG-1    11
Name: louvain, Length: 15476, dtype: category
Categories (33, object): ['0', '1', '2', '3', ..., '29', '30', '31', '32']
